In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

%matplotlib inline
plt.ion()

In [ ]:
def binary_class(x, x_0):

    classification_vals = []

    for x_i in x:
        if x_i > x_0:

            classification_vals.append(1)

        else:
            
            classification_vals.append(0)

    return np.array(classification_vals)

test_data = [0, 5, 7, 2]

print('Decisions for {}: {}.'.format(test_data, binary_class(test_data, 4)))

In [ ]:
def visualize_classify(endpoints, center):

    test_vals = np.linspace(endpoints[0], endpoints[1], 20)
    decisions = binary_class(test_vals, center)

    fig, ax = plt.subplots()
    ax.step(test_vals, decisions)

    cmap, norm = mcolors.from_levels_and_colors(
        [
            endpoints[0],
            center,
            endpoints[1]
        ],
        [
            'red',
            'blue'
        ]
    )

    ax.scatter(test_vals, decisions, c=test_vals, cmap=cmap, norm=norm)
    fig.show()

    return fig, ax

binary_fig, binary_ax = visualize_classify(endpoints=[-5,5],center=3)

In [ ]:
def sigmoid_class(data, weight, bias):

    classification_vals = []

    for x_i in data:

        decision = 1 / (1 + np.exp(-1 * weight * x_i + bias))
        classification_vals.append(decision)

    return np.array(classification_vals)

def visualize_sigmoid(endpoints, bias, weight):

    test_vals = np.linspace(endpoints[0], endpoints[1], 20)
    decisions = sigmoid_class(test_vals, weight, bias)

    fig, ax = plt.subplots()

    ax.plot(test_vals, decisions)

    cmap, norm = mcolors.from_levels_and_colors(
        [
            endpoints[0],
            -bias,
            endpoints[1]
        ],
        [
            'red',
            'blue'
        ]
    )

    ax.scatter(test_vals, np.round(decisions), c=test_vals, cmap=cmap, norm=norm)
    fig.show()

    return fig, ax

sig_fig, sig_ax = visualize_sigmoid(endpoints=[-5, 5], bias=-2, weight=1)

In [ ]:
def calculate_grad(weight, bias, x_vals, y_vals):

    sig_vals = sigmoid_class(x_vals, weight, bias)

    abs_error = np.subtract(sig_vals, y_vals)

    dweight = np.sum(((2 * x_vals * np.exp(-1 * weight * x_vals + bias)) / (1 + np.exp(-1 * weight *x_vals + bias))**3) - ((2 * y_vals * x_vals * np.exp (-1 * weight *x_vals + bias)) / (1 + np.exp(-1 * weight * x_vals + bias))**2)) / len(x_vals)
    dbias = np.sum(((-2 * np.exp(-1 * weight * x_vals + bias)) / (1 + np.exp(-1 * weight * x_vals + bias))**3) + ((2 * y_vals * np.exp(-1 * weight * x_vals + bias)) / (1 + np.exp(-1 * weight * x_vals + bias))**2)) / len(x_vals)

    mse = np.sum(abs_error ** 2) / len(abs_error)

    return dweight, dbias, mse

def training(control_weight, control_bias, left_xlim=-5, right_xlim=5, train_iterations=100):

    fig, (ax, err_ax) = plt.subplots(2, 1)

    ax.set_title('Predicted Sigmoid')
    err_ax.set_title('Error (MSE)')

    fig.tight_layout()

    x = np.linspace(left_xlim, right_xlim, 50)[:, np.newaxis]
    y = sigmoid_class(x, control_weight, control_bias) + np.random.uniform(-0.1, 0.1, x.shape)
    ax.scatter(x, y)

    w = 4
    b = 1

    learning_rate = 5.0

    dw, db, err = calculate_grad(w, b, x, y)

    err_vals = [err]
    y_predicted = sigmoid_class(x, w, b)
    z, = ax.plot(x, y_predicted, color='green')
    e, = err_ax.plot(np.arange(0, len(err_vals)), err_vals)

    ax.set_ylim(bottom=-1)
    err_ax.set_ylim(bottom=0)
    err_ax.set_xlim(left=0, right=train_iterations)

    for i in range(train_iterations):

        w = w - (learning_rate * dw)
        b = b - (learning_rate * db)

        dw, db, err = calculate_grad(w, b, x, y)
        y_predicted = sigmoid_class(x, w, b)
        err_vals.append(err)

        z.set_ydata(y_predicted)
        e.set_xdata(np.arange(0, len(err_vals)))
        e.set_ydata(err_vals)

        fig.canvas.draw()
        fig.canvas.flush_events()

    print('Learned weights after {} iterations are {}. The control weight was {}.'.format(
        train_iterations,
        w,
        control_weight,
    ))

    print('Learned bias after {} iterations is {}. The control bias was {}.'.format(
        train_iterations,
        b,
        control_bias,
    ))

    return fig, ax

train_fig, train_ax = training(1.5, 2.0, train_iterations=100)